In [1]:
import pandas as pd
import numpy as np

In [2]:
genres_dataset = pd.read_json('MasterSongList.json')

In [3]:
genres_dataset.head(10)


,_id,album,artist,audio_features,context,decades,genres,lyrics_features,moods,name,new_context,picture,recording_id,sub_context,yt_id,yt_views
0,{'$oid': '52fdfb440b9398049f3d7a8c'},Gangnam Style (강남스타일),PSY,"[11, 0.912744, 0.083704, 132.069, 0.293137, 0....",[work out],[],[pop],"[oppa, gangnam, style, gangnam, style, najeneu...","[energetic, motivational]",Gangnam Style (강남스타일),work out,http://images.musicnet.com/albums/073/463/405/...,50232.0,[working out: cardio],9bZkp7q19f0,2450112089
1,{'$oid': '52fdfb3d0b9398049f3cbc8e'},Native,OneRepublic,"[6, 0.7457039999999999, 0.11995499999999999, 1...",[energetic],[2012],[pop],"[lately, i, ve, been, i, ve, been, losing, sle...",[happy],Counting Stars,energetic,http://images.musicnet.com/albums/081/851/887/...,5839.0,[energy boost],hT_nvWreIhg,1020297206
2,{'$oid': '52fdfb420b9398049f3d3ea5'},Party Rock Anthem,LMFAO,"[5, 0.709932, 0.231455, 130.03, 0.121740999999...","[energetic, energetic, energetic, energetic]",[],[],"[party, rock, yeah, woo, let, s, go, party, ro...","[happy, celebratory, rowdy]",Party Rock Anthem,housework,http://images.musicnet.com/albums/049/414/127/...,52379.0,"[energy boost, pleasing a crowd, housework, dr...",KQ6zr6kCPj8,971128436
3,{'$oid': '52fdfb410b9398049f3d1eac'},Gentleman,PSY,"[3, 0.705822, 0.053292, 126.009, 0.126016, 0.0...","[party, party, party, party, party, party]",[2010s],[dance],"[alagamun, lan, weh, wakun, heya, hanun, gon, ...","[happy, energetic, celebratory]",Gentleman,energetic,http://images.musicnet.com/albums/082/950/461/...,12353.0,"[driving in the left lane, energy boost, girls...",ASO_zypdnsQ,892096527
4,{'$oid': '52fdfb400b9398049f3d0b19'},On The Floor,Jennifer Lopez,"[3, 0.741757, 0.07277399999999999, 129.985, 0....","[party, party]",[2000s],[reggaeton],"[j, lo, the, other, side, out, my, mine, it, s...",[energetic],On The Floor,work out,http://images.musicnet.com/albums/050/131/765/...,29502.0,"[working out: cardio, dance party: sweaty]",t4H_Zoh7G5A,873285189
5,{'$oid': '52fdfb3e0b9398049f3cdd7f'},The Lazy Song,Bruno Mars,"[8, 0.733856, 0.093043, 174.952, 0.05813699999...",[just woke up],[],[],"[today, i, don, t, feel, like, doing, anything...","[happy, sprightly]",The Lazy Song,just woke up,http://images.musicnet.com/albums/051/655/149/...,46206.0,[waking up on the right side of the bed],fLexgOxsZu0,775630683
6,{'$oid': '52fdfb3f0b9398049f3ce0f0'},Rolling In The Deep,Adele,"[8, 0.7773749999999999, 0.054104, 104.946, 0.0...","[relax, relax, relax]",[],[r&b: soul],"[there, s, a, fire, starting, in, my, heart, r...",[warm],Rolling In The Deep,chillout,http://images.musicnet.com/albums/047/744/109/...,9154.0,"[pleasing a crowd, lying low on a sunday after...",rYEDA3JcQqw,745960922
7,{'$oid': '52fdfb440b9398049f3d7a7d'},Call Me Maybe,Carly Rae Jepsen,"[7, 0.585564, 0.10829699999999999, 120.014, 0....",[work out],[],[pop],"[i, threw, a, wish, in, the, well, don, t, ask...","[energetic, motivational]",Call Me Maybe,work out,http://images.musicnet.com/albums/064/604/149/...,35484.0,[working out: cardio],fWNaR-rxAic,743689468
8,{'$oid': '52fdfb400b9398049f3d0d47'},Somebody That I Used To Know (Feat. Kimbra),Gotye,"[0, 0.418212, 0.105322, 129.054, 0.045461, 0.5...",[candlelit dinner],[],[],"[now, and, then, i, think, of, when, we, were,...","[seductive, nocturnal]",Somebody That I Used To Know (Feat. Kimbra),candlelit dinner,http://images.musicnet.com/albums/056/945/335/...,41786.0,[romantic evening],8UVNT4wvIGY,702996459
9,{'$oid': '52fdfb400b9398049f3d0dc7'},What Makes You Beautiful,One Direction,"[4, 0.81403, 0.079196, 124.991, 0.07244, 0.005...","[studying / working, studying / working, study...",[2010s],[pop],"[don, t, know, what, for, you, re, turning, he...","[happy, celebratory]",What Makes You Beautiful,housework,http://images.musicnet.com/albums/064/019/497/...,20475.0,"[energy boost, pleasing a crowd, housework, wo...",QJO3ROT-A4E,697513708


In [4]:
## Preprocessing data and categorise 

def preprocess_data():
    global filtered_genres_dataset
    genres_dataset.loc[:, 'genres'] = genres_dataset['genres'].apply(''.join)
    genres_dataset.loc[:, 'genres'] = genres_dataset['genres'].apply(consolidate_genre)
    
    dance_dataset = genres_dataset.loc[(genres_dataset['genres'] == 'dance')].sample(n=1200)
    jazz_dataset = genres_dataset.loc[(genres_dataset['genres'] == 'jazz')].sample(n=1200)
    country_dataset = genres_dataset.loc[(genres_dataset['genres'] == 'country')].sample(n=1200)
    filtered_genres_dataset = pd.concat([dance_dataset, jazz_dataset, country_dataset])
                             
def consolidate_genre(genre):
    if (len(genre) > 0):
        return genre.split(':')[0]
    else:
        return genre
    
preprocess_data()


In [5]:
filtered_genres_dataset.head(10)

,_id,album,artist,audio_features,context,decades,genres,lyrics_features,moods,name,new_context,picture,recording_id,sub_context,yt_id,yt_views
13273,{'$oid': '52fdfb3f0b9398049f3ce3b3'},Glowing (The Remixes),Nikki Williams,"[5, 0.813925, 0.097372, 126.044, 0.04627000000...","[party, party, party]",[],dance,"[this, is, my, confessional, seven, seconds, t...","[visceral, aggressive, rowdy]",Glowing (Fedde Le Grand Remix),work out,http://images.musicnet.com/albums/081/121/493/...,58356.0,"[driving in the left lane, working out: cardio...",sQl3wRcRzQE,194680
6354,{'$oid': '52fdfb430b9398049f3d4e9d'},Promises,Nero,"[9, 0.9327139999999999, 0.547392, 143.957, 0.0...","[party, party]",[2012],dance,"[you, got, me, so, wild, how, can, i, ever, de...",[energetic],Promises (Calvin Harris Remix),work out,http://images.musicnet.com/albums/066/193/873/...,27774.0,"[working out: cardio, dance party: sweaty]",u2Uif9N5dac,1284609
6342,{'$oid': '52fdfb3f0b9398049f3ce4ef'},Blind (6-Track Maxi-Single),Hercules & Love Affair,"[9, 0.9191579999999999, 0.060850999999999995, ...",[staying up all night],['00s electronic],dance,"[as, a, child, i, knew, that, the, stars, coul...","[energetic, cold, nocturnal]",Blind (Frankie Knuckles Remix),NaN,http://images.musicnet.com/albums/014/235/225/...,58509.0,NaN,NZ6sLFDUPWg,1291050
875,{'$oid': '52fdfb400b9398049f3d0e23'},4,Beyoncé,"[6, 0.8109759999999999, 0.064914, 117.999, 0.2...","[party, party, party]",[2000s],dance,"[come, take, my, hand, i, won, t, let, you, go...","[cocky, rowdy]",End Of Time,party,http://images.musicnet.com/albums/064/033/735/...,63654.0,"[driving in the left lane, grinding at a night...",OJBfv9CHlcw,28808800
31160,{'$oid': '52fdfb410b9398049f3d1da0'},Horse Meat Disco III,Mungolian Jet Set,[],"[party, party]",[2000s],dance,[],"[energetic, cold]",Moon Jocks 'n' Prog Rocks (Todd Terje's Schlon...,party,http://images.musicnet.com/albums/055/208/207/...,65566.0,"[staying up all night, dance party: sweaty]",5U_I3rdMWrE,3370
1640,{'$oid': '52fdfb400b9398049f3d032b'},Ultra Dance 13,Fedde Le Grand,"[1, 0.8549319999999999, 0.142147, 128.0, 0.061...",[work out],[],dance,"[so, much, love, i, got, so, much, love, to, g...",[energetic],So Much Love,work out,http://images.musicnet.com/albums/063/646/881/...,17791.0,[working out: cardio],JBhBbxm4nak,12541897
17172,{'$oid': '52fdfb3f0b9398049f3cee2e'},Young Soul Rebels,El Coco,"[0, 0.6254000000000001, 0.503181, 116.02, 0.05...",[energetic],[1970s],dance,"[let, s, get, it, get, it, together, let, s, g...","[funky, warm]",Let's Get It Together,energetic,http://images.musicnet.com/albums/048/523/865/...,28996.0,[energy boost],22gMAMT_-dw,83772
28597,{'$oid': '52fdfb460b9398049f3dae74'},Accelerator 3.0,ManMadeMan,"[8, 0.724923, 0.40351000000000004, 145.002, 0....","[party, party, party]",[2000s],dance,[],[hypnotic],Karahana (Altom Remix),coding,http://images.musicnet.com/albums/035/917/801/...,77655.0,"[working out: cardio, dance party: sweaty, cod...",iaSqM9ssUwQ,6681
724,{'$oid': '52fdfb3d0b9398049f3cc393'},Parallel Lines: Deluxe Collector's Edition,Blondie,[],[housework],[1970s],dance,"[once, i, had, a, love, and, it, was, a, gas, ...","[celebratory, campy, lush]",Heart Of Glass,housework,http://images.musicnet.com/albums/014/516/661/...,12580.0,[housework],WGU_4-5RaxU,35799658
8089,{'$oid': '52fdfb460b9398049f3da3e5'},Reggae Gold 2004,Beenie Man,"[5, 0.671504, 0.210581, 94.038, 0.293247, 0.19...","[party, party, party, party, party]",[2010s],dance,"[you, want, a, proper, fix, call, me, you, wan...","[cocky, visceral]",Dude (Remix) (Feat. Ms Thing & Shawwna),work out,http://images.musicnet.com/albums/016/217/957/...,46058.0,"[grinding at a nightclub, working out: weight ...",Z_P0J9L00XI,739786


In [6]:
### Setting up data frame
audio_feature_headers = ['key', 
                         'energy', 
                         'liveliness', 
                         'tempo', 
                         'speechiness', 
                         'acousticness', 
                         'instrumentalness', 
                         'time_signature', 
                         'duration', 
                         'loudness', 
                         'valence', 
                         'danceability', 
                         'mode', 
                         'time_signature_confidence', 
                         'tempo_confidence', 
                         'key_confidence', 
                         'mode_confidence',
                         'moods',
                         'new_context',
                        ] 
audio_features_list = []
for audio_feature in filtered_genres_dataset['audio_features']:
    audio_features_list.append(audio_feature)

for index, mood in enumerate(filtered_genres_dataset['moods']):
    audio_features_list[index].append(mood)
    
for index, context in enumerate(filtered_genres_dataset['new_context']):
    audio_features_list[index].append(context)
    
df_features = pd.DataFrame(audio_features_list, columns=audio_feature_headers)

In [7]:
filtered_genres_dataset['genres'].reset_index(drop=True, inplace=True)
df_features['genres'] = filtered_genres_dataset['genres']


In [8]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

df_features.dropna(inplace=True)
df_features['moods'] = label_encoder.fit_transform(df_features['moods'])
df_features['new_context'] = label_encoder.fit_transform(df_features['new_context'])
labels_finals = df_features['genres']
df_features = df_features.drop(['genres', 
                                'mode_confidence',
                                'time_signature',
                                'duration',
                                'time_signature_confidence', 
                                'tempo',
                                'speechiness', 
                                'key', 
                                'key_confidence',
                                'tempo_confidence',
                                'liveliness',
                                'valence',
                               ], axis=1)

In [9]:
df_features.head(5)

,energy,acousticness,instrumentalness,loudness,danceability,mode,moods,new_context
0,0.813925,0.001439,0.000000,248.04000,0.382521,0.609751,63,17
1,0.932714,0.000037,0.002301,256.41333,0.353901,0.489743,25,17
3,0.810976,0.032820,0.000517,223.98667,0.779680,0.711382,19,12
5,0.854932,0.009598,0.066143,330.00000,0.078987,0.772581,25,17
6,0.6254,0.366983,0.000567,291.80553,0.926459,0.742443,37,8


In [10]:
from sklearn.preprocessing import StandardScaler
standard_scaler = StandardScaler()
df_scaled_features = standard_scaler.fit_transform(df_features)

audio_feature_headers = [
                         'instrumentalness',
                         'acousticness',
                         'moods',
                         'new_context',
                         'mode',
                         'danceability',
                         'energy', 
                         'loudness',
                         ]
df_features_final = pd.DataFrame(df_scaled_features, columns=audio_feature_headers)

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(algorithm='auto', leaf_size=30,
           metric_params=None, n_jobs=1, n_neighbors=3, p=2)

X_train, X_test, y_train, y_test = train_test_split(df_features_final,
                                                    labels_finals, 
                                                    test_size = 0.33, 
                                                    random_state = 101)
knn.fit(X_train, y_train)
pred = knn.predict(X_test)
np.mean(cross_val_score(knn, df_features_final, labels_finals, cv=10))


/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


0.91635030089752978

In [12]:
# Evaluate accuracy
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from nltk import ConfusionMatrix
print(ConfusionMatrix(list(y_test), list(pred)))
print(accuracy_score(y_test, pred))
print(classification_report(y_test, pred))

        |   c         |
        |   o         |
        |   u   d     |
        |   n   a   j |
        |   t   n   a |
        |   r   c   z |
        |   y   e   z |
--------+-------------+
country |<206>  5  12 |
  dance |  23<250>  6 |
   jazz |  24   8<271>|
--------+-------------+
(row = reference; col = test)

0.903105590062
             precision    recall  f1-score   support

    country       0.81      0.92      0.87       223
      dance       0.95      0.90      0.92       279
       jazz       0.94      0.89      0.92       303

avg / total       0.91      0.90      0.90       805



In [13]:
import matplotlib.pyplot as plt
%matplotlib inline

### 